In [1]:

import sys
!{sys.executable} -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Web driver scroll to bottom functionality

import time

# Function to scroll to the bottom
def scroll_to_bottom(driver):
    old_position = driver.execute_script("return window.pageYOffset;")
    while True:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for page to load (adjust time as necessary)
        time.sleep(1)
        # Calculate new scroll position and compare with last scroll position
        new_position = driver.execute_script("return window.pageYOffset;")
        if new_position == old_position:
            break
        old_position = new_position

In [9]:
# Scrape and save original companies

%pip install selenium
%pip install bs4
%pip install supabase
%pip install python-dotenv

from bs4 import BeautifulSoup
from selenium import webdriver
from supabase import create_client, Client
from collections import defaultdict
from dotenv import load_dotenv

import time
import os

load_dotenv()

url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

options = webdriver.ChromeOptions()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

x_descriptions, y_descriptions = set(), set()
x_company_dict, y_company_dict = defaultdict(list), defaultdict(list)

batches = [season + year for year in [str(i).zfill(2) for i in range(5, 24)] for season in ["S", "W"]]

for batch in batches:
    
    driver.get(f"https://www.ycombinator.com/companies?batch={batch}")

    scroll_to_bottom(driver)

    html = driver.page_source

    soup = BeautifulSoup(html, "html.parser")
    
    raw_companies = soup.find_all("span", class_="_coName_86jzd_453")
    raw_descriptions = soup.find_all("span", class_="_coDescription_86jzd_478")

    descriptions = [str(x.text) for x in list(raw_descriptions)]
    companies = [str(x.text) for x in list(raw_companies)]
    
    filtered_descriptions = []
    filtered_companies = []
    for description, company in zip(descriptions, companies):
        if " for " in description:
            filtered_descriptions.append(description)
            filtered_companies.append(company)
    
    for idx, filtered_description in enumerate(filtered_descriptions):
        split_description = filtered_description.split(" for ")
        x_description, y_description = split_description[0], split_description[1]
        x_descriptions.add(x_description)
        y_descriptions.add(y_description)
        company = filtered_companies[idx]
        x_company_dict[x_description].append(company)
        y_company_dict[y_description].append(company)
        original_company_data = {
            "company": company,
            "x_description": x_description,
            "y_description": y_description
        }
        response = supabase.table("original_companies").insert(original_company_data).execute()

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
S05
num companies
8
W05
num companies
0
S06
num companies
11
W06
num companies
7
S07
num companies
19
W07
num companies
13
S08
num companies
22
W08
num companies
21
S09
num companies
26
W09
num companies
16
S10
num companies
36
W10
num companies
27
S11
num companies
60
W11
num companies
45
S12
num companies
82
W12
num companies
66
S13
num companies
52
W13
num companies
46
S14
num companies
80
W14
num companies
74
S15
num

RemoteProtocolError: <ConnectionTerminated error_code:ErrorCodes.NO_ERROR, last_stream_id:19999, additional_data:None>

In [51]:
# Batch read and writes to supabase

def batch_read():
    # Initialize pagination variables
    batch_size = 1000
    page = 0
    all_data = []

    while True:
        response = supabase.table('original_companies').select('company, x_description, y_description').range(page * batch_size, (page + 1) * batch_size - 1).execute()
        
        data = response.data
        if len(data) == 0:
            break
        
        all_data.extend(data)
        
        # Increment the page
        page += 1
    
    return all_data

def batch_write(data, table_name='combinations', batch_size=2000):
    total_rows = len(data)
    total_batches = (total_rows + batch_size - 1) // batch_size

    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, total_rows)
        batch_data = data[start_idx:end_idx]

        supabase.table(table_name).insert(batch_data).execute()
        print(f"Batch {batch_num + 1} of {total_batches} written")

In [52]:
# Generate and save combinations

load_dotenv()

url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

data = batch_read()
company = []
x_description_list = []
y_description_list = []

for row in data:
    company.append(row['company'])
    x_description_list.append(row['x_description'])
    y_description_list.append(row['y_description'])

combinations = []
for idx_x, x_description in enumerate(x_description_list):
    for idx_y, y_description in enumerate(y_description_list):
        if idx_x != idx_y:
            x_company = company[idx_x]
            y_company = company[idx_y]
            combination_data = {
                "company_x": x_company,
                "company_y": y_company,
                "x_description": x_description,
                "y_description": y_description,
            }
            combinations.append(combination_data)

batch_write(combinations)

Batch 1 of 1524 written
Batch 2 of 1524 written
Batch 3 of 1524 written
Batch 4 of 1524 written
Batch 5 of 1524 written
Batch 6 of 1524 written
Batch 7 of 1524 written
Batch 8 of 1524 written
Batch 9 of 1524 written
Batch 10 of 1524 written
Batch 11 of 1524 written
Batch 12 of 1524 written
Batch 13 of 1524 written
Batch 14 of 1524 written
Batch 15 of 1524 written
Batch 16 of 1524 written
Batch 17 of 1524 written
Batch 18 of 1524 written
Batch 19 of 1524 written
Batch 20 of 1524 written
Batch 21 of 1524 written
Batch 22 of 1524 written
Batch 23 of 1524 written
Batch 24 of 1524 written
Batch 25 of 1524 written
Batch 26 of 1524 written
Batch 27 of 1524 written
Batch 28 of 1524 written
Batch 29 of 1524 written
Batch 30 of 1524 written
Batch 31 of 1524 written
Batch 32 of 1524 written
Batch 33 of 1524 written
Batch 34 of 1524 written
Batch 35 of 1524 written
Batch 36 of 1524 written
Batch 37 of 1524 written
Batch 38 of 1524 written
Batch 39 of 1524 written
Batch 40 of 1524 written
Batch 41 